In [1]:
import numpy as np
import pandas as pd
import json
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pickle as pkl
import imblearn
np.random.seed(1332)
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import seaborn as sns
import statsmodels.api as sm
%matplotlib inline
import random
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sys                                               #### Filling the missing values using KNN because the dataset is very less
from impyute.imputation.cs import fast_knn
import imblearn                                      ### sampling the data using the smote method
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import sklearn
import xgboost
from xgboost import XGBClassifier              ### XGBoost classification method
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from xgboost import cv

In [2]:
excel = pd.ExcelFile('Coronna Data CERTAIN with KVB edits.xlsx')
df = pd.read_excel(excel, 'BL+3M')

In [3]:
df = df.drop(columns = ['SubjectID', 'CDate', 'Match'])    ### Unique values
df = df.drop(columns = ['SubjectID.1', 'UNMC_id.1', 'CDate.1', 'init_group.1', 'grp.1', 'UNMC_id.2', 'grp.2', 'init_group.2', 'CDate.1', 'futime.1'])    ### Duplicate values
df = df.drop(columns = ['init_group', 'futime'])    ### same values for all rows

In [4]:
df = df.drop(columns = ['statin_use', 'rfstatus_impute', 'ccpstatus_impute', 'statin_use.1', 'smkyrs', 'numcigs'])  ### Dropping columns who have null values greater than 70%

In [5]:
df = df.drop(columns = ['DAS28-CRP 3m', 'DAS28-CRP BL'])     ### deleting the regression based variables

In [6]:
df = df.rename(columns={"usresultsIgA.1": "usresultsIgA_BL", "usresultsIgG.1": "usresultsIgG_BL", "usresultsIgM.1": "usresultsIgM_BL", 'seatedbp1.1': 'seatedbp1_BL',
                              'seatedbp2.1': 'seatedbp2_BL', 'pres_mtx.1': 'pres_mtx_BL', 'pres_arava.1': 'pres_arava_BL', 'pres_azulfidine.1': 'pres_azulfidine_BL', 
                              'pres_plaquenil.1': 'pres_plaquenil_BL', 'pres_imuran.1': 'pres_imuran_BL', 'pres_minocin.1': 'pres_minocin_BL', 'pres_pred.1': 'pres_pred_BL',
                              'statin_use.1': 'statin_use_BL', 'tender_jts_28.1': 'tender_jts_28_BL', 'BLswollen_jts_28': 'swollen_jts_28_BL',
                              'BLmd_global_assess': 'md_global_assess_BL', 'BLpt_global_assess': 'pt_global_assess_BL', 'BLdi': 'di_BL', 'BLpt_pain': 'pt_pain_BL', 'BLusresultsCRP': 'usresultsCRP_BL',
                              'DAS28-CRP BL': 'DAS28-CRP_BL', })     ### Renaming some columns. Not sure why - did it because previous student did it

In [7]:
df = df.drop('UNMC_id',1)
final_df = df
model_label = LabelEncoder()
final_df['grp'] = model_label.fit_transform(final_df['grp'].astype('str'))
final_df['gender'] = model_label.fit_transform(final_df['gender'].astype('str'))
final_df['final_education'] = model_label.fit_transform(final_df['final_education'].astype('str'))
final_df['race_grp'] = model_label.fit_transform(final_df['race_grp'].astype('str'))
final_df['newsmoker'] = model_label.fit_transform(final_df['newsmoker'].astype('str'))
final_df['drinker'] = model_label.fit_transform(final_df['drinker'].astype('str'))
final_df['ara_func_class'] = model_label.fit_transform(final_df['ara_func_class'].astype('str'))

/state/partition1/job-15710768/ipykernel_9777/2061900728.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('UNMC_id',1)


In [8]:
final_df_1 = final_df[final_df['3MResponse'] != 'Unknown']### We don't need unknown category in our classification data so removing that category
#new_df = final_df[final_df['3MResponse'] == 'Unknown']
final_df_1['3MResponse'] = model_label.fit_transform(final_df_1['3MResponse'].astype('str'))

/state/partition1/job-15710768/ipykernel_9777/839304026.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_1['3MResponse'] = model_label.fit_transform(final_df_1['3MResponse'].astype('str'))


In [9]:
final_df_1

,3MResponse,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,seatedbp1,seatedbp2,pres_mtx,pres_arava,...,tender_jts_28_BL,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL
0,0,0,0.000000,NaN,NaN,NaN,134.0,90.0,1.0,0.0,...,5,3,35.0,10.0,0.00,10.0,4.41,284.0,1640.0,94.0
1,2,1,0.814954,0.714954,0.714954,1.000000,112.0,72.0,1.0,0.0,...,3,3,35.0,17.0,0.25,25.0,1.19,207.0,1010.0,56.0
2,1,1,4.582632,2.869955,4.482632,0.640239,120.0,70.0,1.0,0.0,...,2,6,40.0,10.0,0.00,5.0,0.40,190.0,657.0,100.0
3,1,1,0.015811,NaN,NaN,NaN,120.0,70.0,1.0,0.0,...,3,9,55.0,50.0,0.25,60.0,1.00,44.0,948.0,51.0
4,0,2,0.683486,0.583486,0.583486,1.000000,128.0,80.0,1.0,0.0,...,14,13,65.0,20.0,0.25,65.0,0.25,155.0,824.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,1,2,0.379131,0.279131,0.279131,1.000000,132.0,78.0,1.0,0.0,...,0,9,70.0,15.0,0.00,65.0,6.02,125.0,1000.0,77.0
292,1,1,0.000000,NaN,NaN,NaN,140.0,70.0,0.0,0.0,...,8,7,55.0,80.0,1.00,80.0,7.33,353.0,859.0,211.0
293,2,2,0.000000,NaN,NaN,NaN,126.0,66.0,1.0,0.0,...,2,6,35.0,50.0,0.00,15.0,22.36,266.0,1330.0,253.0
294,0,2,0.000000,NaN,NaN,NaN,120.0,58.0,1.0,0.0,...,5,5,35.0,15.0,0.00,15.0,0.92,163.0,1500.0,44.0


In [9]:
def KNN(new_df):
    sys.setrecursionlimit(100000) 
    imputed_training=fast_knn(new_df.values, k=15)
    return imputed_training
imputed_training = KNN(final_df_1)
c = list(final_df_1.columns)
df_without_nulls = pd.DataFrame(imputed_training, columns=c)

In [25]:
X = df_without_nulls.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
y = df_without_nulls['3MResponse']
#X_train_norm, X_val_norm, Y_train, Y_val = train_test_split(X, y, test_size=0.1, random_state = 351) 

/state/partition1/job-15710768/ipykernel_9777/1212960752.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = df_without_nulls.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [26]:
#X_val_norm['y'] = Y_val

In [27]:
#new_df['3MResponse'] = new_df['3MResponse'].replace('Unknown', np.nan)

/state/partition1/job-15710768/ipykernel_9777/3023486207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['3MResponse'] = new_df['3MResponse'].replace('Unknown', np.nan)


In [28]:
#X_val_norm.isnull().sum()

grp                    0
Type I IFN activity    0
IFNβ activity final    0
IFNα activity final    0
IFNβ/α ratio final     0
                      ..
usresultsCRP_BL        0
usresultsIgA_BL        0
usresultsIgG_BL        0
usresultsIgM_BL        0
y                      0
Length: 78, dtype: int64

In [29]:
#X_train_norm['3MResponse'] = Y_train

In [30]:
#X_train_norm.astype({'3MResponse': 'int32'}).dtypes

grp                    float64
Type I IFN activity    float64
IFNβ activity final    float64
IFNα activity final    float64
IFNβ/α ratio final     float64
                        ...   
usresultsCRP_BL        float64
usresultsIgA_BL        float64
usresultsIgG_BL        float64
usresultsIgM_BL        float64
3MResponse               int32
Length: 78, dtype: object

In [31]:
#combined_train_df = pd.concat([X_train_norm, new_df])

In [32]:
#combined_train_df.dtypes

grp                    float64
Type I IFN activity    float64
IFNβ activity final    float64
IFNα activity final    float64
IFNβ/α ratio final     float64
                        ...   
usresultsCRP_BL        float64
usresultsIgA_BL        float64
usresultsIgG_BL        float64
usresultsIgM_BL        float64
3MResponse             float64
Length: 78, dtype: object

In [33]:
# def KNN(new_df):
#     sys.setrecursionlimit(100000) 
#     imputed_training=fast_knn(new_df.values, k=15)
#     return imputed_training
# imputed_training = KNN(combined_train_df)
# c = list(combined_train_df.columns)
# unknown_combined = pd.DataFrame(imputed_training, columns=c)

In [34]:
# for i in range(len(unknown_combined['3MResponse'])):
#     if unknown_combined['3MResponse'][i] <= 0.5: 
#         unknown_combined['3MResponse'][i] = 0
#     elif unknown_combined['3MResponse'][i] >0.5 and unknown_combined['3MResponse'][i]<=1.5:
#         unknown_combined['3MResponse'][i] = 1
#     else:
#         unknown_combined['3MResponse'][i] = 2

In [17]:
unknown_combined

,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,seatedbp1,seatedbp2,pres_mtx,pres_arava,pres_azulfidine,...,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL,3MResponse
0,1.0,2.966271,1.052589,2.424268,0.434189,153.000000,79.000000,1.000000,1.000000,0.000000,...,2.0,40.0,70.0,1.000,70.0,31.71,179.0,761.0,55.0,0.0
1,0.0,1.900473,1.852377,1.800473,1.028828,120.000000,62.000000,1.000000,0.000000,0.000000,...,19.0,90.0,88.0,0.750,88.0,18.83,230.0,1290.0,47.0,0.0
2,2.0,2.095485,0.786182,1.995485,0.393980,102.000000,82.000000,0.000000,0.000000,0.000000,...,6.0,40.0,50.0,0.875,80.0,40.83,512.0,1420.0,246.0,0.0
3,2.0,0.000000,1.329371,1.794696,0.922436,126.000000,83.000000,0.000000,0.000000,0.000000,...,4.0,40.0,35.0,0.500,45.0,1.27,164.0,753.0,66.0,2.0
4,0.0,0.000000,0.879895,1.204910,0.897352,150.000000,70.000000,0.000000,0.000000,0.000000,...,13.0,50.0,45.0,0.500,90.0,2.24,152.0,709.0,300.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,3.0,1.463426,1.363426,1.363426,1.000000,122.303688,72.395852,0.613210,0.283819,0.076582,...,3.0,35.0,80.0,0.375,50.0,0.33,48.0,492.0,43.0,1.0
264,1.0,0.866307,0.766307,0.766307,1.000000,132.201826,73.834313,0.880444,0.169334,0.008040,...,5.0,50.0,75.0,0.625,45.0,0.96,292.0,1550.0,55.0,1.0
265,1.0,0.288376,0.188376,0.188376,1.000000,122.671680,73.627467,0.632972,0.098606,0.149941,...,6.0,25.0,75.0,0.125,27.0,11.87,117.0,780.0,112.0,1.0
266,0.0,2.409918,1.845131,2.309918,0.798786,121.252032,74.253634,0.616222,0.092116,0.153290,...,4.0,30.0,55.0,1.000,25.0,1.21,319.0,851.0,90.0,1.0


In [35]:
# X_unknown = unknown_combined.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set
# y_unknown = unknown_combined['3MResponse']

/state/partition1/job-15710768/ipykernel_9777/2470334778.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = unknown_combined.drop('3MResponse',1)                            ### Dividing the dataframe into X and y set


In [36]:
X_unknown.shape

(268, 77)

In [26]:
# lab = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']
# from scipy import stats
# for i in df_without_nulls.columns:
#     if i not in lab:
#         IQR = df_without_nulls[i].quantile(0.75) - df_without_nulls[i].quantile(0.25)
#         lower_limit = df_without_nulls[i].quantile(0.25) - 1.5 * IQR
#         upr_limit = df_without_nulls[i].quantile(0.75) + 1.5 * IQR
#         upr = df_without_nulls[i] > upr_limit
#         low = df_without_nulls[i] < lower_limit
#         #df_without_nulls = df_without_nulls[(df_without_nulls[i] < upr_limit) and (df_without_nulls[i] > lower_limit)].all()
#         df_without_nulls = df_without_nulls[~low | upr]

In [27]:
# y= final_df.iloc[df_without_nulls.index,0]
# y = model_label.fit_transform(y.astype('str'))

In [37]:
labels = ['grp','gender','final_education','race_grp','newsmoker','drinker','ara_func_class']     ### normalizing the dataset

for i in X.columns:
    if i not in labels:
        mean = X[i].mean()
        std = X[i].std()
        X[i] = (X[i] - mean) / std
        #X_val_norm[i] = (X_val_norm[i] - mean) / std

In [38]:
labels = ['pres_imuran','pres_minocin','num_tnf','num_nontnf','hxunstab_ang','pres_minocin_BL','ethnicity','hxstroke','pres_imuran_BL']    #### This columns has same value for every row so when I divide by mean and std the value goes to infinity
for i in labels:
    X = X.drop(i,1)
    #X_val_norm = X_val_norm.drop(i,1)

/state/partition1/job-15710768/ipykernel_9777/3788427514.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_unknown = X_unknown.drop(i,1)
/state/partition1/job-15710768/ipykernel_9777/3788427514.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_val_norm = X_val_norm.drop(i,1)


In [39]:
#X_unknown['y'] = y_unknown

In [40]:
#X_val_norm['y'] = Y_val

In [41]:
#X_unknown

,grp,Type I IFN activity,IFNβ activity final,IFNα activity final,IFNβ/α ratio final,seatedbp1,seatedbp2,pres_mtx,pres_arava,pres_azulfidine,...,swollen_jts_28_BL,md_global_assess_BL,pt_global_assess_BL,di_BL,pt_pain_BL,usresultsCRP_BL,usresultsIgA_BL,usresultsIgG_BL,usresultsIgM_BL,y
0,2.0,-0.214022,-0.382524,-0.407449,0.045932,-2.662234,-2.547569,-1.639483,-0.316467,-0.227384,...,-0.170203,0.437578,0.372509,0.100177,-1.343227,-0.242361,3.878891,2.026287,-0.612727,1.0
1,3.0,1.951128,2.686130,1.892806,-0.112427,-0.012512,0.472674,0.649334,-0.316467,-0.227384,...,-0.351213,-1.969829,-2.000893,-0.788320,-1.820167,-0.585451,-0.416145,0.067661,-0.571924,1.0
2,1.0,0.731158,1.135665,0.596710,0.045932,2.095221,2.598030,-1.639483,-0.316467,-0.227384,...,-0.713233,-1.167360,-0.241302,0.211239,-0.242599,-0.453680,-0.339448,-0.212142,-0.979957,2.0
3,2.0,-0.011279,-0.772495,-0.665383,0.045932,1.191907,0.472674,0.649334,-0.316467,-0.227384,...,-0.532223,0.972558,0.781716,-0.047906,0.674592,-0.488293,-1.114940,-0.694026,0.679378,2.0
4,2.0,-0.468299,-0.790953,-0.677592,0.045932,-0.012512,-0.198491,0.649334,-0.316467,-0.227384,...,-0.170203,-0.899871,-1.059716,-1.084486,-0.976351,-0.457323,-0.654758,1.249054,0.203339,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,3.0,0.165026,0.226320,-0.004748,0.045932,-0.246265,-0.562646,-0.080433,0.734557,-0.182574,...,-0.894243,-0.899871,1.190924,-0.307051,-0.059160,-0.598203,-1.711473,-1.884746,-0.952755,1.0
264,1.0,-0.128864,-0.245739,-0.316977,0.045932,0.493189,-0.192566,0.375526,0.314208,-0.193053,...,-0.532223,-0.097401,0.986320,0.211239,-0.242599,-0.559947,0.367870,1.404501,-0.789542,1.0
265,1.0,-0.413310,-0.702628,-0.619172,0.045932,-0.239351,-0.395106,-0.041765,0.051546,0.202108,...,-0.351213,-1.434850,0.986320,-0.825341,-0.902976,0.102550,-1.123462,-0.989375,-0.014278,1.0
266,0.0,0.630871,0.607137,0.490165,-0.261472,-0.298383,-0.168679,-0.228680,0.029695,0.535043,...,-0.713233,-1.167360,0.167906,0.988674,-0.976351,-0.544766,0.597961,-0.768641,-0.313503,1.0


In [42]:
X.to_csv('preprocessing_approach_1_with_unknown_notas4thcategory.csv', index = False, )

In [43]:
#X_val_norm.to_csv('preprocessing_approach_1_test.csv', index=False)